In [34]:
import pandas as pd

df = pd.read_csv('https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv')
df.set_index('Object ID')

C:\Users\brian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Object Number,Is Highlight,Is Public Domain,Department,Object Name,Title,Culture,Period,Dynasty,Reign,...,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository
Object ID,,,,,,,,,,,,,,,,,,,,,
1,1979.486.1,False,False,American Decorative Arts,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
2,1980.264.5,False,False,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
3,67.265.9,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
4,67.265.10,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
5,67.265.11,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
6,67.265.12,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/6,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
7,67.265.13,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/7,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
8,67.265.14,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/8,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
9,67.265.15,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/9,11/19/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"


In [35]:
def clean_country_data(df):
    
    import numpy as np
    
    df['Country'] = df['Country'].astype(str)
    df['Country'] = np.where(((np.char.find(df['Country'].str.lower().values.astype(str)," or ")!=-1) | (np.char.find(df['Country'].str.lower().values.astype(str),"probably")!=-1) | 
                              (np.char.find(df['Country'].str.lower().values.astype(str),"?")!=-1) | (np.char.find(df['Country'].str.lower().values.astype(str),"unknown")!=-1) |
                              (np.char.find(df['Country'].str.lower().values.astype(str),"possibly")!=-1) | (np.char.find(df['Country'].str.lower().values.astype(str),"posssibly")!=-1) |
                              (np.char.find(df['Country'].str.lower().values.astype(str),"perhaps")!=-1)),'nan',df['Country'])
    country_list = list(df['Country'])
    for x in range(len(country_list)): 
        token_list = country_list[x].split("|")
        token_list = sorted(token_list)
        token_list = set(token_list)
        country_list[x] = ', '.join(token_list)
    df['Country'] = pd.Series(country_list)
    df['Country'] = np.where(df['Country'] == 'nan', np.nan, df['Country'])

In [36]:
def get_valid_id_list(category):
    
    #Returns list of object IDs that have valid (non-NaN) value in that particular category column
    #Category could be art title, artist name, country, time period, etc.
    import numpy as np
    
    valid_id_list = []
    valid_rows = np.where(df[category].notnull())[0].tolist()
    for x in valid_rows:
        valid_id_list.append(object_id_list[x])
        
    return valid_id_list

In [37]:
def get_wrong_answers(category, chosen_art):
    
    #Get list of Object IDs that have valid (non NaN) value for that category and have the
    #same department (relevant value) as the correct answer
    import numpy as np
    
    dept_valid_id_list = []
    department = df.loc[df['Object ID'] == chosen_art]['Department'].values[0]
    chosen_art_category_value = df.loc[df['Object ID'] == chosen_art][category].values[0]
    dept_valid_rows = np.where((df['Department'] == department) & (df[category] != chosen_art_category_value))[0].tolist()
    for x in dept_valid_rows:
        dept_valid_id_list.append(object_id_list[x])
    relevant_valid_id_list = list(set(dept_valid_id_list).intersection(set(valid_id_dict[category])))
    
    #Get randomy selected wrong answers among the relevant, valid ID list
    import random
    
    w1 = random.choice(relevant_valid_id_list)
    w2 = random.choice(relevant_valid_id_list)
    w3 = random.choice(relevant_valid_id_list)                                                                   
                                                                       
    return w1,w2,w3                         

In [38]:
def choose_category():
    
    import random
    
    category = random.choice(category_list)
    return category

In [39]:
def choose_art(valid_id_list):
    
    import random
    
    random_object_id = random.choice(valid_id_list)
    
    return random_object_id

In [40]:
def generate_question():
    
    category = choose_category()
    chosen_art = choose_art(valid_id_dict[category])
    w1,w2,w3 = get_wrong_answers(category, chosen_art)
    
    return category, chosen_art, w1, w2, w3

In [41]:
#Clean Data
clean_country_data(df)

In [42]:
#Initialize valid ids for all categories
category_list = ['Artist Display Name','Period','Title','Country','Department']
object_id_list = list(df['Object ID'])
valid_id_dict = dict()
for x in category_list:
    valid_id_dict[x] = get_valid_id_list(x)

In [53]:
active = True

# while active:
    
category,chosen_art,w1,w2,w3 = generate_question()
print(category,chosen_art,w1,w2,w3)

#Sometimes get an error that says "Cannot choose from an empty sequence" like the following.
#Need to find out what is causing this error and fix this.

IndexError: Cannot choose from an empty sequence

In [54]:
list(df['Country'].unique())

[nan,
 'Mexico',
 'Guatemala',
 'United States',
 'England',
 'Spain',
 'United States, England',
 'Netherlands',
 'France',
 'Canada',
 'United States, France',
 'England, France',
 'China',
 'Italy',
 'Japan',
 'Germany',
 'Czech Republic',
 'England, Ireland',
 'United States, Ireland',
 'Venezuela',
 'England, Netherlands',
 'England, Belgium',
 'Italy, France',
 'Bermuda',
 'Ireland',
 'India',
 'NEW ZEALAND ROTORUA WHAKAREWAREWA, France',
 'NEW ZEALAND ROTORUA WHAKAREWAREWA, China',
 'Italy, Germany',
 'Lisez et propagez nos Annales: "L\'Immacul‚e"',
 'Bernard Turner, France',
 'France, Harold Copping',
 'Czech Republic, England',
 'France, Belgium',
 'France, England, Belgium',
 'United States, Germany',
 'Germany, France',
 'Scotland',
 'Czech Republic, Germany',
 'Holland',
 'United States, Denmark',
 'United States, Denmark, Finland',
 'United States, China',
 'India, China',
 'United States, Wales',
 'Italy, United States',
 'United States, England, Wales, Scotland',
 'Switz